In [1]:
import pandas as pd # type: ignore
from services.jira_service import JiraService  # seu service acima
from datetime import datetime
from services.jira_parser import parse_issues_to_dataframe_espaco_de_projetos, prepare_dataframe_for_json_export
import json
from datetime import timedelta

jira = JiraService()


In [7]:
full_issues = jira.get_all_issues_from_project("EP")
issues = full_issues.get("issues", [])

for issue in issues:
    if issue['key'] == "EP-45":
        for field in issue['fields']:
            print(f"CAMPO:{field}")
            print(f"VALOR:{issue['fields'][field]}")
            print("-"*100)





CAMPO:statuscategorychangedate
VALOR:2025-06-30T15:22:26.581-0300
----------------------------------------------------------------------------------------------------
CAMPO:fixVersions
VALOR:[]
----------------------------------------------------------------------------------------------------
CAMPO:statusCategory
VALOR:{'self': 'https://tigruposalus.atlassian.net/rest/api/3/statuscategory/3', 'id': 3, 'key': 'done', 'colorName': 'green', 'name': 'Itens concluídos'}
----------------------------------------------------------------------------------------------------
CAMPO:resolution
VALOR:{'self': 'https://tigruposalus.atlassian.net/rest/api/3/resolution/10000', 'id': '10000', 'description': 'O trabalho foi concluído neste item.', 'name': 'Itens concluídos'}
----------------------------------------------------------------------------------------------------
CAMPO:customfield_10112
VALOR:None
------------------------------------------------------------------------------------------------

In [ ]:
for issue in issues:
    print(issue)

In [ ]:
def calcular_tempo_por_status(changelog: dict, data_criacao: datetime) -> dict:
    """
    Calcula o tempo total em cada status com base no changelog da issue.
    Retorna um dicionário com nome do status e total de dias que permaneceu nele.
    """
    from collections import defaultdict

    tempos_por_status = defaultdict(timedelta)
    

    historico = changelog.get("changelog", {}).get("histories", [])
    historico_ordenado = sorted(historico, key=lambda h: h["created"])

    status_anterior = None
    data_anterior = data_criacao

    for item in historico_ordenado:
        data_atual = pd.to_datetime(item["created"])

        for change in item.get("items", []):
            if change["field"] == "status":
                novo_status = change["toString"]

                if status_anterior:
                    tempos_por_status[status_anterior] += data_atual - data_anterior

                status_anterior = novo_status
                data_anterior = data_atual

    # Adiciona tempo até hoje, se ainda estiver em um status
    if status_anterior:
        tempos_por_status[status_anterior] += pd.Timestamp.now(tz=data_anterior.tz) - data_anterior


    # Converte timedelta para dias
    return {status: round(td.total_seconds() / 86400, 1) for status, td in tempos_por_status.items()}

issue = jira.get_issue_with_changelog(issue_key) # type: ignore
print(issue)


data_criacao = pd.to_datetime(issue["fields"]["created"])
resultado = calcular_tempo_por_status(issue, data_criacao)

print("Tempo em cada status (em dias):")
for status, dias in resultado.items():
    print(f"{status}: {dias} dias")

In [ ]:
for i, issue in enumerate(issues["issues"]):
    if issue['key'] == "EP-38":
        print(issue.get("fields", {}).get("customfield_10278", {}).get("value"))
        print(f"{issue}")
        break

In [ ]:

for i, issue in enumerate(issues["issues"]):
    if issue['fields']['status']['name'] == "Backlog":
        print(f"Título - {issue['fields']['summary']}")

    # if issue["key"] == "EP-3":
    #     print(f"ID - {issue['id']}")
    #     print(f"Tipo - {issue['fields']['issuetype']['name']}")
    #     print(f"Chave - {issue['key']}")
    #     print(f"Título - {issue['fields']['summary']}")
    #     print(f"Prioridade - {issue['fields']['priority']['name']}")
    #     print(f"Descrição - {issue['fields']['description']['content'][0]['content'][0]['text']}")
    #     print(f"Aprovador por (diretor) - {issue['fields']['customfield_10250']}")
    #     print(f"Beneficios Esperados - {issue['fields']['customfield_10248']['content'][0]['content'][0]['text']}")
    #     print(f"Status - {issue['fields']['status']['name']}")
    #     print(f"Grupo Solicitante - {issue['fields']['customfield_10083']['value']}")
    #     print(f"Departamento solicitante - {issue['fields']['customfield_10245']['value']}")
    #     print(f"Solicitante - {issue['fields']['customfield_10093']['content'][0]['content'][0]['text']}")
    #     print(f"Responsável - {issue['fields']['assignee']['displayName']}")
    #     print(f"Telefone do Solicitante - {issue['fields']['customfield_10246']}")
    #     print(f"Email do Solicitante - {issue['fields']['customfield_10247']}")
    #     print(f"Relator - {issue['fields']['creator']['displayName']}")
    #     print(f"Categoria - {issue['fields']['labels'][0]}")
    #     print(f"Estimativa original - {issue['fields']['timeoriginalestimate']}")
    #     print(f"Controle de tempo - {issue['fields']['timetracking']['timeSpent']}")
    #     print(f"Investimento Esperado - {issue['fields']['customfield_10249']}")
    #     print(f"Target start - {issue['fields']['customfield_10022']}")
    #     print(f"Target end - {issue['fields']['customfield_10023']}")
    #     print(f"Data de criação - {issue['fields']['created']}")
    #     print(f"Data de atualização - {issue['fields']['updated']}")
    #     print(f"Data de término - {issue['fields']['resolutiondate']}")


 




